### Repeated Prisioner's Dilemma

- There are two agents that interact in repeated prisioner's dilemma. 
- Actions are: defect (D/0) or cooperate (C/1).
- Defect is dominant strategy, and (D,D) is Nash Eqbm of stage game. 
- Sustained Cooperation is best for everyone in repeated game. 
- Rewards for agents are 

    - (1,1) for (D,D) 
    - (3,0) for (D,C) 
    - (0,3) for (C,D) 
    - (2.5,2.5) for (C,C)

- There is just one state that does change
- There is "discounting". Agents value future profits - they do not seek to maximize discounted sum. 

### Import Packages

In [1]:
from itertools import count
import torch
import math
import torch.optim as optim 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams['font.size']=20

### Helper Functions

In [57]:
def gameReward(a1,a2):
    if (a1 == 0 and a2 == 0): # DD
        return (1,1)
    if (a1 == 0 and a2 == 1): # DC
        return (3,0)
    if (a1 == 1 and a2 == 0): # CD
        return (0,3)
    if (a1 == 1 and a2 == 1): # CC
        return (2.9,2.9)
    
import random
from collections import namedtuple, deque
Transition = namedtuple('Transition',('state1', 'state2', 'next_state1', 'next_state2','a1','a2','r1','r2'))

class ReplayMemory(object):
    # Holds transitions from experience and gives a random batch of transitions for training
    def __init__(self,capacity):
        self.memory = deque([], maxlen=capacity)
        
    def push(self,*args):
        "save transition"
        self.memory.append(Transition(*args))
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):
    # Input: State
    # Output: 0,1
    # Network is going to try to predict the expected Return of taking each action given the INPUT
    def __init__(self, n_obs, n_actions):
        super(DQN,self).__init__()
        self.layer1 = nn.Linear(n_obs, 128)
        self.layer2 = nn.Linear(128,128)
        self.layer3 = nn.Linear(128, n_actions)
    
    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)   

def select_action(state, policy_net, idx):
    # Given state selects action either from the DQN or randomly. 
    global steps_done1, steps_done2
    sample = random.random()
    if idx == 0:
        eps_threshold = EPS_END + (EPS_START-EPS_END)*math.exp(-1.*steps_done1/EPS_DECAY)
        steps_done1 += 1
    else:
        eps_threshold = EPS_END + (EPS_START-EPS_END)*math.exp(-1.*steps_done2/EPS_DECAY)
        steps_done2 += 1
    
    if sample>eps_threshold:
        with torch.no_grad():
            #try: 
            #    return policy_net(state).max(1)[1].view(1,1) # pick up action with largest expected reward
            #except:
            #    return policy_net(state).max(0)[1].view(1,1) # pick up action with largest expected reward
            a = torch.tensor([0,1])
            p = torch.exp(policy_net(state)/BETA)/torch.sum(torch.exp(policy_net(state)/BETA))
            index = p.multinomial(num_samples=1, replacement=True)
            b = a[index]
            return torch.tensor([[b]],dtype=torch.long)
    else:
        randchoice = random.choice([0,1])
        return torch.tensor([[randchoice]],dtype=torch.long)
    
def update_target_net(target_net, policy_net):
    target_net_state_dict = target_net.state_dict()
    policy_net_state_dict = policy_net.state_dict()
    for key in policy_net_state_dict:
        target_net_state_dict[key] = policy_net_state_dict[key]*TAU+target_net_state_dict[key]*(1-TAU)
    target_net.load_state_dict(target_net_state_dict)

def optimize_model(policy_net,target_net,idx,optimizer):
    if len(memory)<BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    if idx == 1:
        action_batch = torch.cat(batch.a1)
        reward_batch = torch.cat(batch.r1)
         # we begin with a batch of transitions - (state, action, reward)
        state_batch = torch.cat(batch.state1)
        # final state is after simulation is done
        # these are the "s(t+1)" from the transitions
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state1)),dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state1 if s is not None])
    elif idx == 2:
        action_batch = torch.cat(batch.a2)
        reward_batch = torch.cat(batch.r2)
         # we begin with a batch of transitions - (state, action, reward)
        state_batch = torch.cat(batch.state2)
        
        # final state is after simulation is done
        # these are the "s(t+1)" from the transitions
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state2)),dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state2 if s is not None])
    
    # For these batch states we compute optimal policy according to our policy-network
    # Compute Q(s,a) and get actions - these are the actions that would have been taken for each batch state 
    state_action_values = policy_net(state_batch.reshape(-1,1)).gather(1,action_batch)
    
    # Compute V(s_t+1)
    next_state_values = torch.zeros(BATCH_SIZE)
    with torch.no_grad():
        # Use the "old" target_net to obtain value (Expected Return) on optimal actions from sampled s(t+1)
        # This is Q_old(s',a')
        next_state_values[non_final_mask] = target_net(non_final_next_states.reshape(-1,1)).max(1)[0]
        
    # Here we have add r + max Q_old(s',a')
    expected_state_action_values = (next_state_values*GAMMA)+reward_batch

    # Compute the loss
    # Q_new(s,a) - r + max Q_old(s',a'): is the error
    # Huber loss function
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values,expected_state_action_values.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(policy_net.parameters(),100)
    optimizer.step()    
    print(f'Loss Agent{idx}:{loss.item()}')

### HyperParameters and Initializations

In [77]:
BATCH_SIZE = 512 # number of transitions sampled from replay buffer
GAMMA = 0.99 # Discount factor
EPS_START = 0.999 # Initial value of Epsilon
EPS_END = 0.05 # End value of Epsilon
EPS_DECAY = 1000 # controls decay rate of Epsilon
TAU = 0.0001 # update rate of target network
LR = 1e-5 # Learning rate of Adam
BETA = 1 # temperature of the greedy-exploratory policy
BETA_DECAY = 1 # temperature of the greedy-exploratory policy

n_obs = 1
n_actions = 2
policy_net1 = DQN(n_obs,n_actions)
target_net1 = DQN(n_obs,n_actions)
policy_net2 = DQN(n_obs,n_actions)
target_net2 = DQN(n_obs,n_actions)
steps_done1 = 0
steps_done2 = 0
optimizer1 = optim.AdamW(policy_net1.parameters(),lr=LR,amsgrad=True)
optimizer2 = optim.AdamW(policy_net2.parameters(),lr=LR,amsgrad=True)
memory=ReplayMemory(10000)

### Training Loop

In [78]:
state1 = torch.tensor(1,dtype=torch.float32).unsqueeze(0)
state2 = torch.tensor(1,dtype=torch.float32).unsqueeze(0)

actions = []
epochs = 10000
for t in range(epochs):
    BETA = 5*0.999**t
    print('\n Iteration:',t)
    # Take action
    a1 = select_action(state1, policy_net1, 1)
    a2 = select_action(state2, policy_net2, 2)

    # Obtain Rewards
    r1, r2 = gameReward(int(a1),int(a2))
    r1 = torch.tensor([r1])
    r2 = torch.tensor([r2])

    # Compute next state
    next_state1 = torch.tensor(a2.item(),dtype=torch.float32).unsqueeze(0)
    next_state2 = torch.tensor(a1.item(),dtype=torch.float32).unsqueeze(0)

    # store memory in transition
    memory.push(state1, state2, next_state1, next_state2, a1,a2,r1,r2)

    # move to next state
    state1 = next_state1
    state2 = next_state2
    
    # optimize
    optimize_model(policy_net1,target_net1, 1, optimizer1)
    optimize_model(policy_net2,target_net2, 2, optimizer2)

    # soft update target_net
    update_target_net(target_net1, policy_net1)
    update_target_net(target_net2, policy_net2)
    
    # record actions
    actions.append([a1.item(),a2.item()])
    
    with torch.no_grad():
        temp1 = torch.tensor(int(0),dtype=torch.float32).unsqueeze(0)
        temp2 = torch.tensor(int(1),dtype=torch.float32).unsqueeze(0)
        print(policy_net1(temp1))
        print(policy_net1(temp2))
        print(policy_net2(temp1))
        print(policy_net2(temp2))


 Iteration: 0
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 1
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 2
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 3
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 4
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 5
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 6
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 7
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 8
tensor([ 0.0291, -0.0596])
tensor

tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 76
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 77
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 78
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 79
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 80
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 81
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 82
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 83
tensor([ 0.0291, -0.0596])
tensor([-0.01

tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 158
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 159
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 160
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 161
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 162
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 163
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 164
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 165
tensor([ 0.0291, -0.0596])
tenso

tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 228
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 229
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 230
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 231
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 232
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 233
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 234
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 235
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor(

tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 308
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 309
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 310
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 311
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 312
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 313
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 314
tensor([ 0.0291, -0.0596])
tensor([-0.0133,  0.0044])
tensor([0.1919, 0.1534])
tensor([0.3408, 0.2762])

 Iteration: 315
tensor([ 0.0291, -0.0596])
tenso

...

Loss Agent1:0.3395501375198364
Loss Agent2:0.3455800414085388
tensor([1.2609, 0.6110])
tensor([1.6168, 0.8346])
tensor([1.3007, 0.6725])
tensor([1.6475, 0.8522])

 Iteration: 5454
Loss Agent1:0.32466045022010803
Loss Agent2:0.3090111315250397
tensor([1.2609, 0.6110])
tensor([1.6168, 0.8347])
tensor([1.3007, 0.6725])
tensor([1.6475, 0.8522])

 Iteration: 5455
Loss Agent1:0.3634968400001526
Loss Agent2:0.3495226502418518
tensor([1.2609, 0.6111])
tensor([1.6168, 0.8347])
tensor([1.3007, 0.6724])
tensor([1.6475, 0.8522])

 Iteration: 5456
Loss Agent1:0.34645548462867737
Loss Agent2:0.3263261318206787
tensor([1.2609, 0.6111])
tensor([1.6168, 0.8348])
tensor([1.3007, 0.6724])
tensor([1.6475, 0.8522])

 Iteration: 5457
Loss Agent1:0.35231468081474304
Loss Agent2:0.30248525738716125
tensor([1.2609, 0.6111])
tensor([1.6169, 0.8348])
tensor([1.3006, 0.6724])
tensor([1.6475, 0.8521])

 Iteration: 5458
Loss Agent1:0.3086884319782257
Loss Agent2:0.3837597668170929
tensor([1.2610, 0.6111])
tensor([1

Loss Agent2:0.32917213439941406
tensor([1.2609, 0.6125])
tensor([1.6180, 0.8370])
tensor([1.3010, 0.6735])
tensor([1.6501, 0.8550])

 Iteration: 5504
Loss Agent1:0.317126601934433
Loss Agent2:0.30682307481765747
tensor([1.2609, 0.6126])
tensor([1.6180, 0.8371])
tensor([1.3010, 0.6736])
tensor([1.6502, 0.8550])

 Iteration: 5505
Loss Agent1:0.3243415355682373
Loss Agent2:0.29769206047058105
tensor([1.2609, 0.6126])
tensor([1.6180, 0.8372])
tensor([1.3010, 0.6735])
tensor([1.6502, 0.8551])

 Iteration: 5506
Loss Agent1:0.317167192697525
Loss Agent2:0.3043915331363678
tensor([1.2609, 0.6127])
tensor([1.6180, 0.8372])
tensor([1.3010, 0.6735])
tensor([1.6502, 0.8551])

 Iteration: 5507
Loss Agent1:0.32407140731811523
Loss Agent2:0.37296879291534424
tensor([1.2609, 0.6127])
tensor([1.6180, 0.8372])
tensor([1.3010, 0.6735])
tensor([1.6503, 0.8551])

 Iteration: 5508
Loss Agent1:0.32418015599250793
Loss Agent2:0.2950915992259979
tensor([1.2609, 0.6127])
tensor([1.6180, 0.8372])
tensor([1.3009,

Loss Agent2:0.3402386009693146
tensor([1.2604, 0.6135])
tensor([1.6181, 0.8387])
tensor([1.3005, 0.6730])
tensor([1.6513, 0.8549])

 Iteration: 5560
Loss Agent1:0.3541550636291504
Loss Agent2:0.3870282769203186
tensor([1.2604, 0.6136])
tensor([1.6181, 0.8388])
tensor([1.3005, 0.6731])
tensor([1.6514, 0.8550])

 Iteration: 5561
Loss Agent1:0.3221686780452728
Loss Agent2:0.34697645902633667
tensor([1.2604, 0.6137])
tensor([1.6182, 0.8389])
tensor([1.3006, 0.6731])
tensor([1.6515, 0.8551])

 Iteration: 5562
Loss Agent1:0.3265535831451416
Loss Agent2:0.3176097571849823
tensor([1.2604, 0.6138])
tensor([1.6182, 0.8390])
tensor([1.3006, 0.6732])
tensor([1.6516, 0.8552])

 Iteration: 5563
Loss Agent1:0.3279151916503906
Loss Agent2:0.32166588306427
tensor([1.2604, 0.6138])
tensor([1.6182, 0.8391])
tensor([1.3006, 0.6732])
tensor([1.6517, 0.8553])

 Iteration: 5564
Loss Agent1:0.32222577929496765
Loss Agent2:0.33070874214172363
tensor([1.2604, 0.6139])
tensor([1.6182, 0.8392])
tensor([1.3006, 0.

Loss Agent2:0.27978742122650146
tensor([1.2608, 0.6154])
tensor([1.6192, 0.8414])
tensor([1.3001, 0.6734])
tensor([1.6525, 0.8561])

 Iteration: 5606
Loss Agent1:0.33371540904045105
Loss Agent2:0.3640545606613159
tensor([1.2608, 0.6155])
tensor([1.6193, 0.8414])
tensor([1.3001, 0.6733])
tensor([1.6525, 0.8561])

 Iteration: 5607
Loss Agent1:0.36818185448646545
Loss Agent2:0.37520831823349
tensor([1.2609, 0.6155])
tensor([1.6194, 0.8415])
tensor([1.3000, 0.6733])
tensor([1.6525, 0.8561])

 Iteration: 5608
Loss Agent1:0.3518410325050354
Loss Agent2:0.30636149644851685
tensor([1.2609, 0.6156])
tensor([1.6195, 0.8415])
tensor([1.3000, 0.6733])
tensor([1.6524, 0.8561])

 Iteration: 5609
Loss Agent1:0.32237595319747925
Loss Agent2:0.34368160367012024
tensor([1.2610, 0.6156])
tensor([1.6196, 0.8416])
tensor([1.3000, 0.6733])
tensor([1.6524, 0.8562])

 Iteration: 5610
Loss Agent1:0.30386245250701904
Loss Agent2:0.3409017324447632
tensor([1.2610, 0.6156])
tensor([1.6197, 0.8416])
tensor([1.2999

RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

### Evaluate Networks (Convergence and Value of states)

##### Agent 1

In [79]:
# When opponent played D/0
state = torch.tensor(int(0),dtype=torch.float32).unsqueeze(0)
policy_net1(state), target_net1(state)

(tensor([1.2611, 0.6157], grad_fn=<AddBackward0>),
 tensor([0.1263, 0.0633], grad_fn=<AddBackward0>))

In [80]:
BETA = 4

a = torch.tensor([0,1])
p = torch.exp(policy_net1(state)/BETA)/torch.sum(torch.exp(policy_net1(state)/BETA))
index = p.multinomial(num_samples=1, replacement=True)
b = a[index]
b

tensor([1])

In [81]:
# When opponent played C/1
state = torch.tensor(int(1),dtype=torch.float32).unsqueeze(0)
policy_net1(state), target_net1(state)

(tensor([1.6198, 0.8418], grad_fn=<AddBackward0>),
 tensor([0.1942, 0.1181], grad_fn=<AddBackward0>))

##### Agent 2

In [82]:
# When opponent played D/0
state = torch.tensor(int(0),dtype=torch.float32).unsqueeze(0)
policy_net2(state), target_net2(state)

(tensor([1.2998, 0.6734], grad_fn=<AddBackward0>),
 tensor([0.1641, 0.1045], grad_fn=<AddBackward0>))

In [83]:
# When opponent played C/1
state = torch.tensor(int(1),dtype=torch.float32).unsqueeze(0)
policy_net2(state), target_net2(state)

(tensor([1.6523, 0.8563], grad_fn=<AddBackward0>),
 tensor([0.2385, 0.0881], grad_fn=<AddBackward0>))

### Trace Path

In [1]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams['font.size']=20
plt.plot(moving_average(np.array(actions)[:,0],100), label = 'Agent 1 Actions', marker = 'o', c = 'b')
plt.plot(moving_average(np.array(actions)[:,1],100), label = 'Agent 2 Actions', marker = 'x', c = 'r')
#plt.title('Fraction of Co-operations in 100 games') 
plt.show()

NameError: name 'plt' is not defined